<a href="https://colab.research.google.com/github/cccg8105/notebook_AD/blob/deep_learning/deep_learning/auto_encoders/AE_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Autoencoder

## Instalación de dependencias

In [1]:
pip install torch===1.6.0 torchvision===0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 748.8MB 21kB/s 
     |████████████████████████████████| 5.9MB 16.2MB/s 
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101


## Importar conjunto de datos

In [2]:
# Importar las librerías
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

movies = pd.read_csv("https://raw.githubusercontent.com/cccg8105/deeplearning-az/master/datasets/Part%205%20-%20Boltzmann%20Machines%20(BM)/ml-1m/movies.dat", sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users  = pd.read_csv("https://raw.githubusercontent.com/cccg8105/deeplearning-az/master/datasets/Part%205%20-%20Boltzmann%20Machines%20(BM)/ml-1m/users.dat", sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings  = pd.read_csv("https://raw.githubusercontent.com/cccg8105/deeplearning-az/master/datasets/Part%205%20-%20Boltzmann%20Machines%20(BM)/ml-1m/ratings.dat", sep = '::', header = None, engine = 'python', encoding = 'latin-1')

training_set = pd.read_csv("https://raw.githubusercontent.com/cccg8105/deeplearning-az/master/datasets/Part%205%20-%20Boltzmann%20Machines%20(BM)/ml-100k/u1.base", sep = "\t", header = None)
# se convierte la estructura para el manejo en pytorch
training_set = np.array(training_set, dtype = "int")
test_set = pd.read_csv("https://raw.githubusercontent.com/cccg8105/deeplearning-az/master/datasets/Part%205%20-%20Boltzmann%20Machines%20(BM)/ml-100k/u1.test", sep = "\t", header = None)
# se convierte la estructura para el manejo en pytorch
test_set = np.array(test_set, dtype = "int")


nb_users = int(max(max(training_set[:, 0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1])))

In [3]:
def convert(data):
    new_data = []
    for id_user in range(1, nb_users+1):
        id_movies = data[:, 1][data[:, 0] == id_user]
        id_ratings = data[:, 2][data[:, 0] == id_user]
        ratings = np.zeros(nb_movies)
        ratings[id_movies-1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [4]:
training_set = convert(training_set)
test_set = convert(test_set)

# se convierten los datos a tipo de variable pytorch
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

## Crear la arquitectura de la Red Neuronal

In [5]:
class SAE(nn.Module):
    # constructor
    def __init__(self, ):
        nodosCapaOculta1 = 20
        nodosCapaOculta2 = 10
        nodosCapaOculta3 = 20

        super(SAE, self).__init__()
        # Creación de la primera capa con 20 nodos ocultos
        self.fc1 = nn.Linear(nb_movies, nodosCapaOculta1)
        # Creación de la segunda capa con 10 nodos ocultos
        self.fc2 = nn.Linear(nodosCapaOculta1, nodosCapaOculta2)
        # Creación de la tercera capa con 20 nodos ocultos
        self.fc3 = nn.Linear(nodosCapaOculta2, nodosCapaOculta3)
        # Creación de la capa final
        self.fc4 = nn.Linear(nodosCapaOculta3, nb_movies)
        # funcion de activacion
        self.activation = nn.Sigmoid()
    
    #
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

In [6]:
sae = SAE()
# funcion de perdidas
criterion = nn.MSELoss()
# optimizador
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

## Entrenamiento

In [ ]:
# cantidad de epocas
nb_epoch = 200

for epoch in range(1, nb_epoch+1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae.forward(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            # la media no es sobre todas las películas, sino sobre las que realmente ha valorado
            mean_corrector = nb_movies/float(torch.sum(target.data > 0)+1e-10) 
            loss.backward()
            train_loss += np.sqrt(loss.data*mean_corrector) ## sum(errors) / n_pelis_valoradas
            s += 1.
            optimizer.step()
    print("Epoch: "+str(epoch)+", Loss: "+str(train_loss/s))

## Evaluar conjunto test 

In [ ]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0:
        output = sae.forward(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        # la media no es sobre todas las películas, sino sobre las que realmente ha valorado
        mean_corrector = nb_movies/float(torch.sum(target.data > 0)+1e-10) 
        test_loss += np.sqrt(loss.data*mean_corrector) ## sum(errors) / n_pelis_valoradas
        s += 1.

print("Test Loss: "+str(test_loss/s))